## RIDE_UCOA_to_dictionary

E. Quinn 1/1/2020

Reads the RIDE UCOA expense data and builds a dictionary for the EG data

The dictionary keys are the six UCOA classifiers: Fund, Loc, Func, Sub, JC, and Obj

## Import standard python datascience packages

In [1]:
import math
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from datetime import *
from datascience import *

In [3]:
pd.set_option("display.max_rows",1000)               #set pandas display options
pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 50)

### Show the directory we are running in

In [4]:
!pwd

/home/gquinn/EG/school_committee/finance_subcommittee/notebooks


### Read the consolidated 2010-2018 RIDE data

In [5]:
aexp  = pd.read_csv("../allexp.csv")                           #read the .csv file with 2010-2018 for all LEAs
EGexp = aexp.loc[aexp['Dist No']==90.0]                        #just select EG
print('aexp: ',aexp.shape)                                     #show the number of rows and columns
print(aexp.columns)                                            #show the column names
print('EGexp: ',EGexp.shape)                                   #show the rows and columns for just EG                     
EGexp[['Year','Fund','Loc','Func','Prog',\
       'Sub','Obj','JC','Actual','Budget']].head()             #show the first few rows for selected columns

/home/gquinn/.conda/envs/data8a/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,7,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


aexp:  (2135261, 25)
Index(['Unnamed: 0', 'ID', 'Dist No', 'District Name', 'Fund',
       'Fund Description', 'Loc Type', 'Loc Type desc', 'Loc',
       'Location Description', 'Func', 'Function Description', 'Prog',
       'Program Description', 'Sub', 'Subject Description', 'Obj',
       'Object Description', 'JC', 'Job Class Description', 'Actual', 'Budget',
       'School Enrollment', 'Year', 'District'],
      dtype='object')
EGexp:  (24980, 25)


,Year,Fund,Loc,Func,Prog,Sub,Obj,JC,Actual,Budget
136300,2018,10000000.0,5106.0,111.0,10.0,1700.0,52710.0,1100.0,5464.55,4762.30
136301,2018,10000000.0,5106.0,111.0,10.0,1500.0,52710.0,1100.0,4530.19,4136.87
136302,2018,10000000.0,5106.0,111.0,10.0,500.0,52710.0,1100.0,4272.80,3950.66
136303,2018,10000000.0,5106.0,111.0,10.0,1900.0,52710.0,1100.0,4232.02,3861.71
136304,2018,10000000.0,4103.0,111.0,10.0,1700.0,52710.0,1100.0,4090.86,3656.02


### Convert pandas dataframe to a dictionary keyed by the six UCOA categorical variables 

In [6]:
EGd = EGexp.to_dict()

In [7]:
EG_ucoa = {}

for key in EGd['Obj'].keys():                                #all columns have same key values
    Obj = EGd['Obj'][key]
    if (Obj not in EG_ucoa.keys()):                          #add keys to dictionary the first time
        EG_ucoa[Obj] = {}                                    #they are encountered
    Loc = EGd['Loc'][key]
    if (Loc not in EG_ucoa[Obj].keys()):
        EG_ucoa[Obj][Loc] = {}
    Func = EGd['Func'][key]
    if (Func not in EG_ucoa[Obj][Loc].keys()):
        EG_ucoa[Obj][Loc][Func] = {}
    Prog   = EGd['Prog'][key]
    if (Prog not in EG_ucoa[Obj][Loc][Func].keys()):
        EG_ucoa[Obj][Loc][Func][Prog] = {}
    Sub   = EGd['Sub'][key]
    if (Sub not in EG_ucoa[Obj][Loc][Func][Prog].keys()):
        EG_ucoa[Obj][Loc][Func][Prog][Sub] = {}
    JC   = EGd['JC'][key]
    if (JC not in EG_ucoa[Obj][Loc][Func][Prog][Sub].keys()):
        EG_ucoa[Obj][Loc][Func][Prog][Sub][JC] = {}
    Fund   = EGd['Fund'][key]
    if (Fund not in EG_ucoa[Obj][Loc][Func][Prog][Sub][JC].keys()):
        EG_ucoa[Obj][Loc][Func][Prog][Sub][JC][Fund] = {}
    Year   = EGd['Year'][key]
    if (Year not in EG_ucoa[Obj][Loc][Func][Prog][Sub][JC][Fund].keys()):
        EG_ucoa[Obj][Loc][Func][Prog][Sub][JC][Fund][Year] = {}
        
    EG_ucoa[Obj][Loc][Func][Prog][Sub][JC][Fund][Year]['Budget'] = EGd['Budget'][key]  #save budget
    EG_ucoa[Obj][Loc][Func][Prog][Sub][JC][Fund][Year]['Actual'] = EGd['Actual'][key]  #save actual

### Build dictionaries for value labels

In [8]:
FundD = {}

for key in EGexp['Fund'].keys():
    Fund = EGexp['Fund'][key]
    if (Fund not in FundD.keys()):
        FundD[Fund] = EGexp['Fund Description'][key]
        
FundD

{10000000.0: 'General Fund',
 21011100.0: 'P.L. 94-142, IDEA Part B - Allocation',
 21011200.0: 'P.L. 94-142, IDEA Pre-School - Allocation',
 21021100.0: 'Title I - Part A - Disadvantaged Youth - Allocation',
 21031100.0: 'Title II - Part A - Teacher Quality - Allocation',
 21041100.0: 'Title III - Language Acquisition - Allocation',
 23671000.0: 'English Language Learner Categorical Funds',
 24012008.0: 'Rhode Island Foundation #1',
 24012063.0: 'Gen Youth Foundation',
 24030018.0: 'Education Development Center, Inc.',
 60010000.0: 'School Food Service Funds',
 21032302.0: 'Title II - Part D - Educational Tech - Competitive - Model Classrooms',
 21091100.0: 'Perkins Vocational Education - Allocation - Secondary - Even Year',
 21413100.0: 'RI Dept of Human Services - Rising Stars',
 23682000.0: 'RIDE State Grant - Award 1 (Leadership)',
 24012000.0: 'Champlin Foundation #1',
 21093100.0: 'Perkins Vocational Education - Targeted - Leadership',
 23612000.0: 'Full Day Kindergarten (State 

In [9]:
FuncD = {}

for key in EGexp['Func'].keys():
    Func = EGexp['Func'][key]
    if (Func not in FuncD.keys()):
        FuncD[Func] = EGexp['Function Description'][key]
        
FuncD

{111.0: 'Instructional Teachers',
 321.0: 'Building Upkeep/Utilities/Maintenance',
 211.0: 'Guidance and Counseling',
 511.0: 'Principals and Assistant Principals',
 113.0: 'Instructional Paraprofessionals',
 531.0: 'Superintendent and School Board',
 331.0: 'Data Processing',
 232.0: 'Therapists, Psychologists et al',
 231.0: 'Program Management',
 332.0: 'Business Operations',
 212.0: 'Library and Media',
 216.0: 'Student Health and Services-Non Instructional',
 512.0: 'School Office',
 213.0: 'Extracurricular',
 532.0: 'Legal',
 222.0: 'Staff Development and Support',
 214.0: 'Student Health and Services',
 313.0: 'Safety',
 112.0: 'Substitutes',
 121.0: 'Pupil-Use Technology and Software',
 122.0: 'Instructional Materials/Trips/Supplies',
 311.0: 'Transportation',
 221.0: 'Curriculum Development',
 241.0: 'Academic Student Assessment',
 431.0: 'Public, Parochial, Private, Charter Pass-Throughs',
 432.0: 'Retiree Benefits and Other',
 312.0: 'Food Service',
 215.0: 'Academic Interve

In [10]:
ProgD = {}

for key in EGexp['Prog'].keys():
    Prog = EGexp['Prog'][key]
    if (Prog not in ProgD.keys()):
        ProgD[Prog] = EGexp['Program Description'][key]
        
ProgD

{10.0: 'Regular Elementary/Secondary Education Programs',
 20.0: 'Special Education',
 90.0: 'Co-Curricular and Extracurricular Activities',
 0.0: 'Other Programs',
 40.0: 'Bilingual/ESL Education',
 13.0: 'PBGR (Performance Based Graduation Requirements)',
 30.0: 'Career and Technical Education Programs',
 50.0: 'Non-Public Schools Programs',
 62.0: 'Summer School',
 14.0: 'Early Intervening Services',
 63.0: 'After School Programs',
 11.0: 'Gifted and Talented',
 18.0: 'Regular Education - High Quality Assessments',
 29.0: 'Special Education - Improving Student Achievement',
 26.0: 'Special Education - Teacher Quality & Equitable Distribution',
 28.0: 'Special Education - High Quality Assessments',
 59.0: 'Non-Public - Improving Student Achievement',
 17.0: 'Regular Education - PK-20 Data Systems',
 27.0: 'Special Education - PK-20 Data Systems',
 16.0: 'Regular Education - Teacher Quality & Equitable Distribution',
 60.0: 'Adult/Continuing Ed, Summer School,  After School'}

In [11]:
JCD = {}

for key in EGexp['JC'].keys():
    JC = EGexp['JC'][key]
    if (JC not in JCD.keys()):
        JCD[JC] = EGexp['Job Class Description'][key]
        
JCD

{1100.0: 'Teachers',
 4700.0: 'Custodial Staff',
 1500.0: 'Counselors and Placement Officers',
 2500.0: 'Principals and Assistant Principals',
 4600.0: 'Aides and Other Non-Certified Staff',
 2100.0: 'Executive',
 4200.0: 'Finance and Administration - Support',
 1700.0: 'Therapists, Nurses, Social Workers, and Psychologists',
 2300.0: 'School Administration',
 4100.0: 'Executive - Support',
 3500.0: 'Other School Activities - Mid Level',
 1600.0: 'Library Professionals',
 4901.0: 'Maintenance and Repair Crew',
 4300.0: 'School Administration - Support',
 2200.0: 'Finance and Administration',
 3200.0: 'Finance and Administration - Mid Level',
 3307.0: 'Director of Athletics',
 4800.0: 'Student Activity Advisors and Coaches',
 0.0: 'None - For non-compensation and benefits Expenditure Accounts',
 2400.0: 'Curriculum and Assessment',
 1300.0: 'Teachers',
 1294.0: 'Long Term Substitute Teacher',
 1295.0: 'Substitute Teachers A',
 4604.0: "Substitute TA's",
 4606.0: 'Resource Teacher Aide',

In [12]:
SubD = {}

for key in EGexp['Sub'].keys():
    Sub = EGexp['Sub'][key]
    if (Sub not in SubD.keys()):
        SubD[Sub] = EGexp['Subject Description'][key]
        
SubD

{1700.0: 'Natural Sciences',
 1500.0: 'Mathematics',
 500.0: 'English Language Arts',
 1900.0: 'Social Sciences',
 700.0: 'Foreign Languages',
 5.0: 'Grade 3',
 2101.0: 'General Education with Supports and Services - School Year (Census Cost Centers 32202 and 32221)',
 3.0: 'Grade 1',
 4.0: 'Grade 2',
 6.0: 'Grade 4',
 1.0: 'Kindergarten',
 7.0: 'Grade 5',
 2103.0: 'Special Class - School Year (Census Cost Centers 32203, 32205, and 32218)',
 1200.0: 'Physical Education and Health',
 2110.0: 'Pre-School, Part-Time - School Year (Census Cost Center 32213)',
 200.0: 'Art',
 2500.0: 'Non-Instruction',
 2000.0: 'Technical Education/Computer Technology',
 800.0: 'Guidance',
 1600.0: 'Music/Theatre and Performing Arts/Drama',
 0.0: 'General Education',
 1800.0: 'Office Occupations',
 2122.0: 'Speech and Language Services - School Year',
 2125.0: 'Occupational Therapy - School Year',
 2400.0: 'Literacy and Reading',
 1300.0: 'Family and Consumer Education',
 2600.0: 'Library Science',
 2121.0:

In [13]:
ObjD = {}

for key in EGexp['Obj'].keys():
    Obj = EGexp['Obj'][key]
    if (Obj not in ObjD.keys()):
        ObjD[Obj] = EGexp['Object Description'][key]
        
ObjD

{52710.0: 'Workers Compensation Premium',
 55401.0: 'Advertising Costs',
 54403.0: 'Telephone',
 51110.0: 'Regular Salaries',
 52102.0: 'Life',
 52103.0: 'Dental',
 52121.0: 'Health and Medical - Self Insured / Active',
 52203.0: 'Teacher/Administrative Pension - ERSRI (Defined Benefit)',
 52207.0: 'Survivor Benefits - ERSRI',
 52208.0: 'MERS Pension (Defined Benefit)',
 52218.0: 'MERS Pension (Defined Contribution)',
 52301.0: 'FICA',
 52302.0: 'Medicare',
 53301.0: 'Professional Development and Training Services',
 53303.0: 'Conferences / Workshops',
 53406.0: 'Other Services',
 53705.0: 'Shipping and Postage',
 53706.0: 'Catering',
 54602.0: 'Rental of Equipment and Vehicles',
 55501.0: 'Printing',
 55503.0: 'Document Copying',
 55803.0: 'Employee Travel - Non-Teachers',
 56101.0: 'General Supplies and Materials',
 56404.0: 'Subscriptions and Periodicals',
 57305.0: 'Equipment',
 58102.0: 'Other Dues and Fees',
 53402.0: 'Legal Services',
 53214.0: 'Mentoring',
 51132.0: 'Department

In [14]:
LocD = {}

for key in EGexp['Loc'].keys():
    Loc = EGexp['Loc'][key]
    if (Loc not in LocD.keys()):
        LocD[Loc] = EGexp['Location Description'][key]
        
LocD

{5106.0: 'East Greenwich High School',
 4103.0: 'Archie R. Cole Middle School',
 3102.0: 'James H. Eldredge El. School',
 3105.0: 'Frenchtown School',
 3107.0: 'Meadowbrook Farms School',
 3108.0: 'George Hanaford School',
 1100.0: 'Superintendents Office',
 2400.0: 'Technology',
 1400.0: 'Student Support Services',
 2100.0: 'Financial',
 2900.0: 'Building Operations',
 0.0: 'East Greenwich School District',
 1.0: 'Main Administration Building',
 1101.0: 'School Committee',
 1200.0: 'Staff Development and Support',
 1300.0: 'Program and Curriculum Development',
 2000.0: 'Other',
 7350.0: 'Warwick School District',
 7420.0: 'The Metropolitan Regional Career and Technical Center',
 7905.0: 'Homebound/Home Instruction',
 7980.0: 'Chariho School District',
 7990.0: 'Foster-Glocester School District',
 8119.0: 'Bradley School - South',
 8151.0: 'Cornerstone School',
 8165.0: 'Eleanor Briggs School',
 8190.0: 'Harmony Hill School',
 8236.0: 'Meeting Street School',
 8241.0: 'Mount Pleasant A

In [15]:
EG_ucoa[57306.]

{3105.0: {122.0: {20.0: {2103.0: {0.0: {10000000.0: {2018: {'Budget': 0.0,
        'Actual': 499.98}}}}},
   10.0: {1.0: {0.0: {10000000.0: {2017: {'Budget': 13376.0,
        'Actual': 12939.03},
       2016: {'Budget': 500.0, 'Actual': 0.0},
       2013: {'Budget': 1000.0, 'Actual': 239.68}},
      23612000.0: {2016: {'Budget': 4443.78, 'Actual': 4443.78}}}},
    3.0: {0.0: {10000000.0: {2017: {'Budget': 800.0, 'Actual': 786.9},
       2016: {'Budget': 500.0, 'Actual': 0.0},
       2013: {'Budget': 1000.0, 'Actual': 974.89}}}},
    4.0: {0.0: {10000000.0: {2017: {'Budget': 300.0, 'Actual': 0.0},
       2016: {'Budget': 500.0, 'Actual': 0.0},
       2013: {'Budget': 1000.0, 'Actual': 965.87}}}},
    5.0: {0.0: {10000000.0: {2013: {'Budget': 1000.0, 'Actual': 0.0}}}}}},
  216.0: {10.0: {2500.0: {0.0: {10000000.0: {2016: {'Budget': 445.0,
        'Actual': 445.0}}}}}},
  512.0: {10.0: {0.0: {0.0: {10000000.0: {2016: {'Budget': 703.0,
        'Actual': 702.74},
       2015: {'Budget': 958